In [ ]:
import socketserver

IPS = {}
STONES = 20
MAX = 4

class NimTCPHandler(socketserver.BaseRequestHandler):

    def my_move(self):
        #TODO WRITE YOUR CODE HERE        
        remains = IPS[self.ip]
        take = 0
        
        
        return take
    
    def handle(self):
        self.ip = self.client_address[0]
        self.request.send(bytes(f"Hello, {self.ip}. We start the game with {STONES} stones, you take 1-{MAX}. How many will you take?\n>", 'utf8'))    
        IPS[self.ip] = STONES
    
        while True:
            
            # read and validate the move of user
            self.data = str(self.request.recv(1024).strip(), 'utf8')
            print(f"[{self.ip}] Client message: {self.data}")
            if self.data not in [str(x) for x in range(1, MAX+1)]:
                self.request.send(bytes("You voilate the rules. Enter another amount\n>", 'utf8'))
                continue
            move = int(self.data)
            if IPS[self.ip] - move < 0:
                self.request.send(bytes("You grab more than you can. Enter another amout\n>", 'utf8'))    
                continue
            
            # make a move of user
            IPS[self.ip] -= move
            
            # test for the end
            if IPS[self.ip] == 0:
                self.request.send(bytes("You lost\n", 'utf8'))
                break
            if IPS[self.ip] == 1:
                self.request.send(bytes("You won\n", 'utf8'))
                break
            
            # if this is not the end
            message = "You took {} stones. {} remains\n".format(move, IPS[self.ip])
            self.request.send(bytes(message, 'utf8'))
            
            
            # make a move!
            my = self.my_move()
            IPS[self.ip] -= my
            message = "Server {} takes {} stones, {} remains\n".format(self.ip, my, IPS[self.ip])
            self.request.send(bytes(message, 'utf8'))
            if IPS[self.ip] == 1:
                self.request.send(bytes("I won!!!", 'utf8'))
                break

In [ ]:
with socketserver.TCPServer(('0.0.0.0', 8989), NimTCPHandler) as server:
    server.serve_forever()